# Runge phenomina

This is NOT yet complete, was supposed to be a quick demonstration, but the current polynomial functionality in use does not support polynomial division when p^E/P^M  with M>E. So, for now we are leaving this alone.

In [2]:
using NLOptControl
using Polynomials
using Plots
using FastGaussQuadrature
pyplot()
#default(guidefont = font(17), tickfont = font(15), legendfont = font(12), titlefont = font(20))

Plots.PyPlotBackend()

In [10]:
# define problem
x₀= Float64(-1); xₙ= Float64(1);  # TODO change x to t and y to x
x = Array(linspace(x₀,xₙ,100));
α₁ =  1; α₂ = 0; α₃ = 25; α₄ =  7;
N = Int64(5); # TODO eventually make it multiple-interval

####################################
# perform analytical calcualtions
####################################
A = Poly([1]);
B = Poly([α₁,α₂,α₃])
γ = div(A,B); 
y = polyval(γ,x)

# evaluate the integral
∫γ = polyint(γ);
Y = polyval(∫γ,x[end]) - polyval(∫γ,x[1]);
C = Y - polyval(∫γ,x[end]); # constant of integration
∫y = polyval(∫γ,x) + C;

# evaluate the derivative
dγ = polyder(γ);
dy = polyval(dγ,x);

####################################
# construct polynomial approximation
####################################
τ, ω = gaussradau(N);

# scale the problem to the interval
xs = scale_tau(τ,x₀,xₙ) # scale the interval
ωₛ = scale_w(ω,x₀,xₙ)   # scale the weights
#TODO check integral matrix
#TODO figure out how to get the last part of the integral!

# approximate the integral
fτ = polyval(γ,xs);
ζ =  cumsum(ωₛ.*fτ,1)

# approximate the derivative --> needed in defect constraints
D = poldif(xs, 1)
dζ = D*fτ

#################
# post processing
#################
fp=plot(0,leg=:false)
plot!(x,y,label=string(γ),w=6)
scatter!(xs,fτ,label=string("ftau with N = ",N),markershape = :hexagon, markersize=10)

dp=plot(0,leg=:false)
plot!(x,dy,label="derivative",w=6)
plot!(xs,dζ,label="approx. derivative",line=(4,:dash))

ip=plot(0,leg=:false)
plot!(x,∫y,label=@sprintf("integral = %0.3f",∫y[end]),w=6)
plot!(xs,ζ,label=@sprintf("approx. integral = %0.3f",ζ[end]),line=(4,:dash))

plot(fp,ip,dp,layout=(3,1))


LoadError: LoadError: MethodError: no method matching /(::Int64, ::Polynomials.Poly{Int64})
Closest candidates are:
  /(::Integer, !Matched::Integer) at int.jl:35
  /(::Real, !Matched::Complex{T<:Real}) at complex.jl:182
  /(::Union{Int16,Int32,Int64,Int8}, !Matched::BigFloat) at mpfr.jl:272
  ...
while loading In[10], in expression starting on line 13

In [3]:
savefig("test2a.png")

# 

In [7]:
A


Poly(1)

In [7]:
savefig("test2b.png")

# 

In [8]:
B

Poly(1 + 25⋅x)

In [9]:
savefig("test2c.png")

# 

In [11]:
savefig("test2d.png")

In [17]:
savefig("test2e.png")